<a href="https://colab.research.google.com/github/sudip234-source/Bangaluru-House-Price-Prediction/blob/main/Bangaluru_House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
hs = pd.read_csv('/content/drive/MyDrive/Data Sets files/Bengaluru_House_Data.csv')
hs.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
hs.shape

(13320, 9)

In [ ]:
hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [ ]:
hs.isna().sum()

,0
area_type,0
availability,0
location,1
size,16
society,5502
total_sqft,0
bath,73
balcony,609
price,0


In [ ]:
hs.describe()

,bath,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [ ]:
hs.drop(columns=['society', 'area_type', 'availability','balcony'], inplace=True)
hs.location = hs.location.fillna('Sarjapur Road')
hs.size = hs['size'].fillna('2 BHK')
hs.bath = hs.bath.fillna(hs.bath.median())

In [ ]:
for column in hs.columns:
  print(hs[column].value_counts())
  print('**********************')

location
Whitefield                        540
Sarjapur  Road                    399
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       234
                                 ... 
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Uvce Layout                         1
Abshot Layout                       1
Name: count, Length: 1306, dtype: int64
**********************
size
2 BHK         5215
3 BHK         4310
4 Bedroom      826
4 BHK          591
3 Bedroom      547
1 BHK          538
2 Bedroom      329
5 Bedroom      297
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
11 Bedroom       2
10 BHK           2
14 BHK           1
13 BHK           1
12 Bedroom       1
27 BHK     

In [ ]:
hs['bhk'] = hs['size'].str.split().str.get(0).astype(int)

In [ ]:
hs.bhk.isna().sum()

0

In [ ]:
hs[hs.bhk>20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [ ]:
hs.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [ ]:
len(hs.total_sqft.str.split('-').str.get(0).str.strip().unique())

1966

In [ ]:
def convert_sqft_to_num(x):
  tokens = x.split('-')
  if len(tokens) == 2:
    return (float(tokens[0])+float(tokens[1]))/2
  try:
    return float(x)
  except:
    return None

In [ ]:
hs.total_sqft = hs.total_sqft.apply(convert_sqft_to_num)

In [ ]:
hs.total_sqft.unique()

array([1056. , 2600. , 1440. , ..., 1258.5,  774. , 4689. ])

In [ ]:
hs['price_per_sqft'] = np.around((hs.price*100000)/(hs.total_sqft),2)
hs.price_per_sqft.sample(5)

,price_per_sqft
6405,2790.70
12985,5681.82
3726,10000.00
1866,5686.27
1347,5785.88


In [ ]:
hs.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678300e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266868e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434305e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311747e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [ ]:
hs.location = hs.location.apply(lambda x:x.strip())
location_count = hs.location.value_counts()
location_count

,count
location,
Whitefield,541
Sarjapur Road,399
Electronic City,304
Kanakpura Road,273
Thanisandra,237
...,...
1Channasandra,1
Hosahalli,1
Vijayabank bank layout,1


In [ ]:
loc_less_10 = location_count[location_count<=10]
loc_less_10

,count
location,
BTM 1st Stage,10
Nagadevanahalli,10
Basapura,10
Sector 1 HSR Layout,10
Dairy Circle,10
...,...
1Channasandra,1
Hosahalli,1
Vijayabank bank layout,1


In [ ]:
hs.location = hs.location.apply(lambda x:'other' if x in location_count[location_count<=10] else x)

In [ ]:
hs.location.value_counts()

,count
location,
other,2886
Whitefield,541
Sarjapur Road,399
Electronic City,304
Kanakpura Road,273
...,...
Nehru Nagar,11
Banjara Layout,11
LB Shastri Nagar,11


In [ ]:
hs.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.81
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.38
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.56
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.89
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.00


Outlier Detection

In [ ]:
hs = hs[np.sqrt(hs.total_sqft/hs.bhk)>=20]

In [ ]:
hs.shape

(11784, 7)

In [ ]:
hs[['price_per_sqft','bhk']].describe()

,price_per_sqft,bhk
count,11784.000000,11784.00000
mean,6180.079407,2.60667
std,4004.261538,0.93104
min,267.830000,1.00000
25%,4207.720000,2.00000
50%,5278.775000,2.00000
75%,6800.000000,3.00000
max,176470.590000,16.00000


In [ ]:
def remove_outliers_sqft(df):
  df_output = pd.DataFrame()
  for key,subdf in hs.groupby('location'):
    m = np.mean(subdf.price_per_sqft)
    st = np.std(subdf.price_per_sqft)
    gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
    df_output = pd.concat([df_output,gen_df],ignore_index = True)
  return df_output

hs=remove_outliers_sqft(hs)
#hs.describe()

In [ ]:
def remove_outliers_bhk(df):
  exclude_indices = np.array([])
  for key,subdf in df.groupby('location'):
    bhk_stats = {}
    for bhk,bhkdf in subdf.groupby('bhk'):
      bhk_stats[bhk] = {'mean':np.mean(hs.price_per_sqft),'std':np.std(hs.price_per_sqft),'count':bhkdf.shape[0]}
    for bhk,bhkdf in subdf.groupby('bhk'):
      stats = bhk_stats.get(bhk-1)
      if stats and stats['count'] > 5:
        exclude_indices = np.append(exclude_indices,bhkdf[bhkdf.price_per_sqft<(stats['mean'])].index.values)
  return df.drop(exclude_indices,axis='index')

In [ ]:
hs = remove_outliers_bhk(hs)
hs.shape

(6239, 7)

In [ ]:
hs.drop(['price_per_sqft','size'],axis=1,inplace=True)


In [ ]:
hs.sample(5)

,location,total_sqft,bath,price,bhk
5744,Sarjapur Road,1550.0,2.0,89.00,3
1177,Bommanahalli,1160.0,2.0,53.00,2
7308,Yelahanka,1175.0,2.0,67.77,2
8149,other,1533.0,3.0,150.00,3
4577,Marathahalli,957.0,2.0,46.90,2


In [ ]:
X = hs.drop(columns=['price'])
y = hs.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline,make_pipeline

In [ ]:
column_trf = make_column_transformer((OneHotEncoder(sparse_output=False),['location']),remainder='passthrough')
scaler = StandardScaler()
lr = LinearRegression()
pipe = make_pipeline(column_trf,scaler,lr)

In [ ]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [ ]:
X_train

,location,total_sqft,bath,bhk
2183,Garudachar Palya,1150.0,2.0,2
6832,Whitefield,1778.0,3.0,3
614,Balagere,1007.0,2.0,2
1768,Electronic City,1128.0,2.0,2
6310,Thubarahalli,1200.0,2.0,2
...,...,...,...,...
5059,Pattandur Agrahara,1550.0,2.0,3
7275,Yelahanka,1823.0,3.0,3
7354,Yelenahalli,1240.0,2.0,2
7706,other,2000.0,3.0,3


In [ ]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [ ]:
from sklearn.metrics import accuracy_score,r2_score
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8013606130376121


Applying Lasso reg
```



In [ ]:
pipe1 = make_pipeline(column_trf,scaler,Lasso(alpha=0.09))
pipe1

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=0.09))])

In [ ]:
pipe1.fit(X_train,y_train)
y_pred_l1 = pipe1.predict(X_test)
r2_score(y_test,y_pred_l1)
#

0.8017855488762249

In [ ]:
pipe2 = make_pipeline(column_trf,scaler,Ridge(alpha=0.5))
pipe2

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.5))])

In [ ]:
pipe2.fit(X_train,y_train)
y_pred_l2 = pipe2.predict(X_test)
r2_score(y_test,y_pred_l2)

0.8013516109172436

In [ ]:
import pickle
pickle.dump(pipe,open('Lasso_model.pkl','wb'))